# Podstawy programowania (AD) 2

## Tomasz Rodak


Wykład VI

# Funkcje jako obiekty. Domknięcia



## [Obiekty pierwszej klasy](https://en.wikipedia.org/wiki/First-class_citizen)

W językach programowania obiekt pierwszej klasy to taki, który:

* może być wartością zmiennej;
* może być argumentem funkcji;
* może być zwrócony z funkcji.

Zamierzeniem projektowym Pythona było, aby **wszystkie** obiekty były obiektami pierwszej klasy.

> One of my goals for Python was to make it so that all objects were "first class." By this, I meant that I wanted   all objects that could be named in the language (e.g., integers, strings, functions, classes, modules, methods,   etc.) to have equal status. That is, they can be assigned to variables, placed in lists, stored in dictionaries,   passed as arguments, and so forth.<br>
*Guido van Rossum*

Funkcje w Pythonie są obiektami pierwszej klasy. Sprawdźmy to.

### Funkcja jako wartość zmiennej

Zmienna wskazująca na funkcję wbudowaną:

```python
>>> wartość_bezwzględna = abs
>>> wartość_bezwzględna(-5)
5
```

Zmienna wskazująca na funkcję zdefiniowaną przez użytkownika:

```python
>>> from functools import reduce
>>> from operator import add
>>> sumator = lambda seq: reduce(add, seq)
>>> sumator([1, 2, 3])
6
>>> sklej = lambda s: ' '.join(s)
>>> sklej(['ala', 'ma', 'kota'])
'ala ma kota'
>>> def signum(x):
...     if x > 0:
...         return 1
...     if x < 0:
...         return -1
...     return 0
>>> znak_liczby = signum
>>> znak_liczby(-3), znak_liczby(0), znak_liczby(23.87)
(-1, 0, 1)
```


Dygresja: co tak naprawdę jest nazwą funkcji? W komórce wyżej zdefiniowaliśmy funkcję `signum()` i utworzyliśmy dla niej alias `znak_liczby`, który jak widać zachowuje się jak funkcja. Czy możemy mówić, że `znak_liczby` jest funkcją i to o takiej właśnie nazwie? 

Tworzenie aliasu jest wygodne, gdyż pozwala na dostosowanie nazwy wywołującej funkcję, jednak dla narzędzi odwołujących się do metadanych funkcji nie jest to jej właściwa nazwa.
Nazwa funkcji przechowywana jest w dwóch w atrybutach:
* `.__name__`
* `.__code__.co_name`:

```python
>>> znak_liczby.__name__
'signum'
>>> znak_liczby.__code__.co_name
'signum'
>>> f = lambda x: 2 * x
>>> f
<function <lambda> at 0x7f87fce14c20>
>>> f.__name__
'<lambda>'
>>> f.__code__.co_name
'<lambda>'
```

Wartości tych atrybutów mogą się różnić, gdyż wartość atrybutu `.__name__` można zmienić na inną. Atrybut `.__code__.co_name` jest tylko do odczytu. Gdy tworzysz funkcję instrukcją `def`, to oba te atrybuty zostają ustawione w funkcji na nazwę występującą po słowie `def`. Ponadto niejawnie wykonywane jest przypisanie tworzące zmienną wskazującą na zdefiniowaną właśnie funkcję. Możesz oczywiście utworzyć dowolną ilość aliasów do funkcji, nie ma to jednak żadnego wpływu na metadane funkcji, w tym na opisane wyżej atrybuty. Więcej informacji na tem temat znajdziesz [tutaj.](https://medium.com/@vadimpushtaev/name-of-python-function-e6d650806c4) Post ten zawiera przykłady dekoratorów -- powiemy o nich w następnym wykładzie.

### Funkcja jako argument funkcji

Chcemy pokazać, że w Pythonie funkcje mogą być argumentami innych funkcji. Jest to temat ciekawy sam w sobie i  posiadający liczne zastosowania, dlatego omówimy go nieco szerzej. Wykorzystamy dyskusję ze strony [Composing Programs](http://composingprograms.com/pages/16-higher-order-functions.html) zawierającej kurs CS1 na uniwersytecie Berkeley.

Funkcje manipulujące innymi funkcjami to **funkcje wyższego rzędu**. Użycie funkcji wyższego rzędu pozwala na implementację wzorców obliczeniowych w oderwaniu od konkretnych wartości. Dzięki temu zmniejsza się ilość powtórzeń w kodzie występujących na poziomie metod, rośnie natomiast czytelność i łatwość utrzymania kodu.

Podane niżej funkcje obliczają skończone sumy różnych ciągów:

```python
>>> def sum_naturals(n):
...     '''Zwraca 1 + 2 + ... + n.'''
...     total = 0
...     for k in range(1, n + 1):
...         total = total + k
...     return total
>>> def sum_cubes(n):
...     '''Zwraca 1**3 + 2**3 + ... + n**3.'''
...     total = 0
...     for k in range(1, n + 1):
...         total = total + k*k*k
...     return total
>>> def sum_inverses(n):
...     '''Zwraca 1/1 + 1/2 + ... + 1/n'''
...     total = 0
...     for k in range(1, n + 1):
...         total = total + 1/k
...     return total
>>> sum_naturals(3)
6
>>> sum_cubes(3)
36
>>> sum_inverses(3)
1.8333333333333333
```

Widać, że na poziomie algorytmu każda z tych funkcji robi to samo, a różnice biorą się jedynie z nazwy i wyrazu ogólnego w aktualizowanej sumie. Każdą z tych funkcji moglibyśmy wygenerować za pomocą szablonu:

<!-- ```python
def <nazwa_funkcji>(n):
    suma = 0
    
    for k in range(1, n + 1):
        suma = suma + <wyraz>(k)
    
    return suma
``` -->
```python
def <nazwa_funkcji>(n):
    total = 0
   
    for k in range(1, n + 1):
        total = total + <wyraz>(k)
   
    return total
```

Interpretując ten szablon jako funkcję widzimy, że nie tylko `n` możemy traktować jako jego parametr ale również i `<wyraz>`. Jakiego rodzaju parametrem powinien być wyraz? Jeśli sumujemy liczby naturalne, to `<wyraz>` ma wartość `k`, jeśli sumujemy sześciany, to `<wyraz>` ma wartość `k**3`. Zatem `<wyraz>` jest funkcją!

Oto implementacja szablonu:

<!-- ```python
def suma_ciągu(n, wyraz):
    '''Zwraca sumę wyraz(1) + wyraz(2) + ... + wyraz(n)'''
    suma = 0
    
    for k in range(1, n + 1):
        suma  = suma + wyraz(k)
        
    return suma
``` -->

```python
>>> def partial_sum(n, term):
...     '''Zwraca sumę term(1) + term(2) + ... + term(n)'''
...     total = 0
...     for k in range(1, n + 1):
...         total = total + term(k)
...     return total
```

Tak zdefiniowana funkcja będzie gotowa do wykonania sumowania, gdy podamy liczbę wyrazów do obliczenia i przepis na wyraz ciągu, czyli parametr `term`:

```python
>>> def identity(k):
...     return k
>>> def cube(k):
...     return k**3
>>> def inverse(k):
...     return 1 / k
>>> partial_sum(5, identity)
15
>>> partial_sum(5, cube)
225
>>> partial_sum(5, inverse)
2.2833333333333333
```

Możemy teraz zdefiniować konkretną funkcję sumującą w oparciu o szablon:

```python
>>> def sum_cubes(n):
...     '''Zwraca 1**3 + 2**3 + ... + n**3.'''
...     return partial_sum(n, cube)
>>> sum_cubes(3)
36
```

**_Ćwiczenie:_** *Zdefiniuj w oparciu o szablon `partial_sum()` funkcje `sum_naturals()` i `sum_inverses()`.* 

### Funkcja jako wartość zwracana z funkcji

W Pythonie funkcja może zostać zwrócona z funkcji. Fakt ten zilustrujemy a zarazem wykorzystamy do jeszcze nieco innego rozwiązania problemu z sumą fragmentu ciągu. 

Wykorzystujemy powtórnie algorytm z szablonem w innej konfiguracji: 
<!-- def fabryka_sumatorów(wyraz):
    '''Zwraca funkcję obliczającą sumy wyraz(1) + wyraz(2) + ... + wyraz(n)'''
    def sumator(n):
        suma = 0

        for k in range(1, n + 1):
            suma  = suma + wyraz(k)

        return suma
    
    return sumator
def kwadrat(x):
    return x ** 2

suma_kwadratów = fabryka_sumatorów(kwadrat) -->
```python
>>> def make_summator(term):
...     '''Zwraca funkcję obliczającą sumy term(1) + term(2) + ... + term(n)'''
...     def summator(n):
...         total = 0
...         for k in range(1, n + 1):
...             total = total + term(k)
...         return total
...     return summator
>>> def sum_squares(n):
...     return make_summator(lambda x: x**2)(n)
>>> sum_squares(3)
14
```

Zauważ, że parametry `n` i `term` występujące na tym samym poziomie w funkcji `partial_sum()` tu zostały rozdzielone. Możesz myśleć, że parametr `term` jest parametrem głównym; parametr `n` przyjął rolę parametru tymczasowego. Funkcja `make_summator()` definiuje nową funkcję w oparciu o argument `term`. Nowa funkcja `summator()` jest funkcją jednej zmiennej `n` zwracającą sumę 

```
term(1) + term(2) + ... + term(n)
```

**_Ćwiczenie:_** *Zdefiniuj trzy omawiane wcześniej konkretne funkcje sumujące wykorzystując wzorzec taki jak przy określaniu funkcji `sum_squares()`.*

**_Ćwiczenie:_** *Przerób funkcję `make_summator()` tak aby teraz parametr `n` był główny a `term` tymczasowy. Zmień, jeśli trzeba, nazwy funkcji. Co zwraca funkcja `make_summator()` po tej zmianie?*


## Domknięcia

Podany w poprzedniej sekcji przykład, po bliższym przyjrzeniu się wydają się stać w sprzeczności z tym co już wiemy na temat lokalnej przestrzeni nazw. W poprzednim wykładzie twierdziłem przecież, że lokalna przestrzeń nazw utworzona podczas wywołania funkcji po zakończeniu jej działania (czyli po zwróceniu wartości) zostaje zapomniana. 

Jak w takim razie działa ten kod?

```python
>>> def make_summator(term):
...     '''Zwraca funkcję obliczającą sumy term(1) + term(2) + ... + term(n)'''
...     def summator(n):
...         total = 0
...         for k in range(1, n + 1):
...             total = total + term(k)
...         return total
...     return summator
>>> sum_squares = make_summator(lambda x: x**2)
>>> sum_squares(3)
14
```

Przebieg zdarzeń:
* Funkcja `make_summator()` zwraca funkcję `summator()`.
* Funkcja `summator()` odnosi się do zmiennej lokalnej `term`.
* Po wykonaniu 
  ```python
  >>> sum_squares = make_summator(lambda x: x**2)
  ```
  `make_summator()` kończy działanie.
* Zmienna `term` przestaje istnieć, gdyż jest to zmienna lokalna.
* Istnieje jednak funkcja `sum_squares()`, która odwołuje się do nie istniejącej/zapomnianej zmiennej `term`.
* __Skąd funkcja `sum_squares()` wie, że zmienna `term` ma wartość `lambda x: x**2`?__

Rozwiązanie tej zagadki tkwi w mechanizmie <a href="https://en.wikipedia.org/wiki/Closure_(computer_programming)"><b>domknięcia</b></a>  (*closure*).

Domknięcia polegają na dołączeniu do funkcji pewnych informacji o stanie środowiska, w którym funkcja została utworzona. W przypadku funkcji `summator()` niezbędna do jej działania jest zewnętrzna wobec jej przestrzeni nazw zmienna `term`. Dlatego zmienna ta zostaje dołączona do zwracanej funkcji. Oczywiście w pamięci zachowany zostaje obiekt, który ta zmienna nazywa.

Dołączone zmienne znajdują się w atrybucie `.__code__.co_freevars`:

```python
>>> sum_squares.__code__.co_freevars
('term',)
```

Atrybut `.__closure__[0].cell_contents` zawiera odpowiadające im wartości.

```python
>>> sum_squares.__closure__[0].cell_contents
<function <lambda> at 0x7f87fce14c20>
```

Wcześniej powiedzieliśmy już, że istnieją zmienne:
* globalne -- tworzone na poziomie modułu,
* lokalne -- tworzone podczas wywołania funkcji.

Jaką w takim razie zmienną z punktu widzenia funkcji `sumator()` jest `wyraz`. Otóż jest to trzeci rodzaj zmiennej -- **zmienna swobodna**. Z dokumentacji Pythona:
>  If a variable is used in a code block but not defined there, it is a free variable.

Z punktu widzenia funkcji `make_summator()` zmienna `term` jest po prostu lokalna. Z punktu widzenia funkcji `summator()` zmienna `term` jest zmienną swobodną. Funkcja `summator()` nie definiuje tej zmiennej, ale jej używa. W momencie wywołania funkcji `make_summator()` zmienna `term` zostaje dołączona do funkcji `summator()`, która staje się funkcją domkniętą.

Więcej szczegółów znajdziesz w oficjalnej [dokumentacji](https://docs.python.org/3/reference/executionmodel.html#naming-and-binding). Przeczytaj też ciekawą dyskusję [stąd.](https://stackoverflow.com/questions/12919278/how-to-define-free-variable-in-python)

### LEGB

Pamiętamy z poprzedniego wykładu, że w Pythonie występują trzy rodzaje przestrzeni nazw: wbudowana, globalna, lokalna. Tworzą one hierarchię przeszukiwania, gdy interpreter poszukuje nazwy zmiennej:

```
lokalna --> globalna --> wbudowana
```
Teraz możemy dopisać do tego schematu jeszcze jedną, pośrednią przestrzeń nazw: zakres dołączony (*enclosed*). Uzupełniona hierarchia przeszukiwania wygląda po tym uzupełnieniu tak:  
```
lokalna --> dołączona --> globalna --> wbudowana
```
Od pierwszych liter
```
Local --> Enclosed --> Global --> Built-in
```
zasada ta nazywana jest [LEGB.](https://sebastianraschka.com/Articles/2014_python_scope_and_namespaces.html#3-legb---local-enclosed-global-built-in) Zasada LEGB decyduje o **zasięgu**, czyli widoczności zmiennej.

### Instrukcja `nonlocal`

Oto ciekawy przykład realizacji domknięcia zaczerpnięty z książki L. Ramahlo:
<!-- def uśredniacz():
    '''Zwraca średnią kroczącą.'''
    wartości = []
    
    def średnia(wartość):
        wartości.append(wartość)
        return sum(wartości) / len(wartości)
    
    return średnia
oceny = uśredniacz()
inne_oceny = uśredniacz()
oceny(5), inne_oceny(4)
oceny(3), inne_oceny(3)
oceny(5), inne_oceny(2) -->
```python
>>> def moving_average():
...     '''Zwraca średnią kroczącą.'''
...     values = []
...     def average(value):
...         values.append(value)
...         return sum(values) / len(values)
...     return average
>>> grades = moving_average()
>>> other_grades = moving_average()
>>> grades(5), other_grades(4)
(5.0, 4.0)
>>> grades(3), other_grades(3)
(4.0, 3.5)
>>> grades(5), other_grades(2)
(4.333333333333333, 3.0)
```

Zmienne swobodne i ich wartości:

```python
>>> grades.__code__.co_freevars
('values',)
>>> grades.__closure__[0].cell_contents
[5, 3, 5]
>>> other_grades.__code__.co_freevars
('values',)
>>> other_grades.__closure__[0].cell_contents
[4, 3, 2]
```

Aby obliczyć średnią wystarczy pamiętać dotychczasową sumę wartości i ich liczbę.
Poprawmy zgodnie z tą sugestią funkcję `uśredniacz()` (bardziej uczenie mówi się: poddajmy refaktoryzacji).

Oto pierwsza próba. Jest błędna. Widzisz błąd?

```python
>>> def moving_average():
...     '''Zwraca średnią kroczącą.'''
...     total, count = 0, 0
...     def average(value):
...         total += value
...         count += 1
...         return total / count
...     return average
```

Zmienne `total` i `count` są przypisane w ciele funkcji `average()`, są to zatem zmienne lokalne! 

```python
>>> grades = moving_average()
>>> grades(5)
---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
...
UnboundLocalError: cannot access local variable 'total' where it is not associated with a value
```

Widzimy analogię między zmienną swobodną a lokalną i zmienną globalną a lokalną. Jeśli zmienna jest przypisana w funkcji, to jest lokalna i **tylko** w zakresie lokalnym będzie poszukiwana. Pamiętamy, że używamy instrukcji `global`, aby zmienna przypisana w ciele funkcji była globalna.

Analogicznie, aby zmienna przypisana w ciele funkcji była swobodna, trzeba zadeklarować ją jako swobodną instrukcją `nonlocal`:
<!-- def uśredniacz():
    '''Zwraca średnią kroczącą.'''
    suma, liczba = 0, 0
    
    def średnia(wartość):
        nonlocal suma, liczba
        suma += wartość
        liczba += 1
        return suma / liczba
    
    return średnia
oceny = uśredniacz()
oceny(5)
oceny(3)
oceny(5)
oceny.__code__.co_freevars
oceny.__closure__[0].cell_contents, oceny.__closure__[1].cell_contents -->

```python
>>> def moving_average():
...     '''Zwraca średnią kroczącą.'''
...     total, count = 0, 0
...     def average(value):
...         nonlocal total, count
...         total += value
...         count += 1
...         return total / count
...     return average
>>> grades = moving_average()
>>> grades(5)
5.0
>>> grades(3)
4.0
>>> grades(5)
4.333333333333333
>>> grades.__code__.co_freevars
('count', 'total')
>>> grades.__closure__[0].cell_contents
3
>>> grades.__closure__[1].cell_contents
13
```